In [16]:
import string
import random

In [6]:
rules = {
        "?*x hello ?*y": [
        "How do you do. Please state your problem."
        ],
        "?*x computer ?*y": [
        "Do computers worry you?",
        "What do you think about machines?",
        "Why do you mention computers?",
        "What do you think machines have to do with your problem?",
        ],
}

In [23]:
def is_variable(pattern):

    return (type(pattern) is str
            and pattern[0] == '?'
            and len(pattern) > 1
            and pattern[1] != '*'
            and pattern[1] in string.ascii_letters
            and ' ' not in pattern)

def is_segment(pattern):

    return (type(pattern) is list
            and pattern
            and len(pattern[0]) > 2
            and pattern[0][0] == '?'
            and pattern[0][1] == '*'
            and pattern[0][2] in string.ascii_letters
            and ' ' not in pattern[0])

In [8]:
def match_pattern(pattern, user_input, bindings=None):

    if bindings is False:
        return False

    if pattern == user_input:
        return bindings

    bindings = bindings or {}

    if is_segment(pattern):
        token = pattern[0] # segment variable is the first token
        var = token[2:] # segment variable is of the form ?*x
        return match_segment(var, pattern[1:], user_input, bindings)
    elif is_variable(pattern):
        var = pattern[1:] # single variables are of the form ?foo
        return match_variable(var, [user_input], bindings)
    elif contains_tokens(pattern) and contains_tokens(user_input):

        return match_pattern(pattern[1:],
                             user_input[1:],
                             match_pattern(pattern[0], user_input[0], bindings))
    else:
        return False

In [18]:
def match_segment(var, pattern, user_input, bindings, start=0):

    if not pattern:
        return match_variable(var, user_input, bindings)

    word = pattern[0]
    try:
        pos = start + user_input[start:].index(word)
    except ValueError:

        return False

    var_match = match_variable(var, user_input[:pos], dict(bindings))
    match = match_pattern(pattern, user_input[pos:], var_match)

    if not match:
        return match_segment(var, pattern, user_input, bindings, start + 1)
    
    return match

In [21]:
user_input = "I am happy."

matching_rules = []

for pattern, transforms in rules.items():
    pattern = pattern.split()
    replacements = match_pattern(pattern, user_input)
    print(replacements)
    if replacements:
        matching_rules.append((transforms, replacements))
        
print(matching_rules)

False
False
[]


In [25]:
is_variable("?*x hello ?*y")

False